In [1]:
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher

# 1) Load Core Data & Model 

df = pd.read_csv('data_cleaned_LDA_final.csv')

# Load the spaCy model
nlp = spacy.load("en_core_web_lg", disable=["tagger", "parser", "ner", "textcat"])
print("spaCy model loaded.")

C:\Users\Belinda\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


spaCy model loaded.


In [3]:
# 2) Build skill list (dedupe + filter + whitelist)

# LOAD O*NET FILES 
df_tech    = pd.read_csv("Technology_Skills.csv", encoding="utf-8")
df_general = pd.read_csv("Skills.csv",            encoding="utf-8")

tech_skills = df_tech["Example"].dropna().astype(str).tolist()
general_skills = df_general["Element Name"].dropna().astype(str).tolist()

# LOAD ESCO FILE AND PROCESS LABELS 
df_esco = pd.read_csv("skills_en.csv", encoding="utf-8", on_bad_lines='skip')

# Get the preferred skill names
esco_preferred = df_esco['preferredLabel'].dropna().astype(str).tolist()

# Get the altLabels, stored as '\n'-separated strings, splits those strings into individual synonyms and flattens the list.
esco_alt_labels = df_esco['altLabels'].dropna().apply(
    lambda x: x.split('\n')
).explode().astype(str).tolist()


# --- COMBINE, DEDUPLICATE, AND CLEAN ALL SOURCES ---

# Combine all lists (O*NET examples + O*NET general + ESCO preferred + ESCO alt labels)
all_skills_raw = tech_skills + general_skills + esco_preferred + esco_alt_labels

# Deduplicate and lowercase the entire combined dictionary
skill_list = list(dict.fromkeys(s.strip().lower() for s in all_skills_raw if s and s.strip()))
print(f"Total unique O*NET + ESCO skills combined: {len(skill_list)}")

Total unique O*NET + ESCO skills combined: 107890


In [4]:
# --- APPLY FILTERING LOGIC ---

MIN_SKILL_LENGTH = 3
SHORT_WHITELIST = {"r","go","c","c#","c++",".net"}
generic_phrases_to_exclude = {
    "instagram","facebook","whatsapp","email","telephone","call",
    "telegram","twitter","youtube","linkedin","zoom", "writing", "monitoring", "speaking"  
}

skill_list_filtered = [
    s for s in skill_list
    if ((len(s.split()) > 1) or (len(s) >= MIN_SKILL_LENGTH) or (s in SHORT_WHITELIST))
    and (s not in generic_phrases_to_exclude)
]
print(f"Final O*NET + ESCO skill list size after filtering: {len(skill_list_filtered)}")

Final O*NET + ESCO skill list size after filtering: 107860


In [5]:
# 3) PhraseMatcher (prefer longest)
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
for s in skill_list_filtered:
    matcher.add(s, [nlp.make_doc(s)])

print(f"PhraseMatcher initialized with {len(matcher)} patterns.")

# 4) Batch extraction
df = df.reset_index(drop=True)
job_texts_full = df["clean_title+desc"].fillna("").astype(str).tolist()
print(f"\nStarting extraction on {len(job_texts_full)} documents...")

extracted = []
for doc in nlp.pipe(job_texts_full, batch_size=256):
    # Get all matches, including overlapping ones
    matches = matcher(doc) 
    
    # Post-process to filter out shorter, overlapping matches 
    # Sort matches by length (e - s) descending to prioritize longer matches
    matches_sorted = sorted(matches, key=lambda x: (x[2] - x[1]), reverse=True)
    
    seen_spans = [] 
    filtered_matches = set()
    
    for match_id, start, end in matches_sorted:
        is_overlapping = False
        # Check if the current match is contained within an already kept and therefore longer match
        for seen_start, seen_end in seen_spans:
            if start >= seen_start and end <= seen_end:
                is_overlapping = True
                break
        
        if not is_overlapping:
            # Keep the longer match and mark its span as seen
            filtered_matches.add(nlp.vocab.strings[match_id])
            seen_spans.append((start, end))

    # Append the sorted list of unique, non-overlapping skills
    extracted.append(sorted(filtered_matches))

df["extracted_skills_list"] = extracted
print("\nO*NET_ESCO-based skill extraction is COMPLETE (Optimized).")

PhraseMatcher initialized with 107860 patterns.

Starting extraction on 65314 documents...


C:\Users\Belinda\anaconda3\Lib\site-packages\spacy\pipeline\lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)



O*NET_ESCO-based skill extraction is COMPLETE (Optimized).


In [6]:
# 5) Overview of extraction
out = "data_with_extracted_skills_ONET_ESCO.csv"
df.to_csv(out, index=False)
print(f"Final dataset saved to '{out}'.")

total_rows = len(df)
matched = sum(bool(x) for x in df["extracted_skills_list"])
print(f"\n--- Skill Matching Statistics (O*NET_ESCO) ---")
print(f"Total documents: {total_rows}")
print(f"Documents with matched skills: {matched}")
print(f"Percentage of matched documents: {matched/total_rows*100:.2f}%")

Final dataset saved to 'data_with_extracted_skills_ONET_ESCO.csv'.

--- Skill Matching Statistics (O*NET_ESCO) ---
Total documents: 65314
Documents with matched skills: 64495
Percentage of matched documents: 98.75%


In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ast

try:
    df['extracted_skills_list'] = df['extracted_skills_list'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) and x.strip().startswith('[') else x
    )
except Exception as e:
    print(f"Warning: literal_eval skipped. Assuming lists already. Error: {e}")

# Build counts
skill_counts = [len(x) for x in df['extracted_skills_list'] if isinstance(x, list)]

if len(skill_counts) == 0:
    print("No skills found to plot. Check the matcher/dictionary and input text.")
else:
    # Basic stats
    mean_count   = float(np.mean(skill_counts))
    median_count = float(np.median(skill_counts))
    stdev_count  = float(np.std(skill_counts))
    p95_count    = float(np.quantile(skill_counts, 0.95))
    p99_count    = float(np.quantile(skill_counts, 0.99))
    nonzero_pct  = 100.0 * (np.sum(np.array(skill_counts) > 0) / len(skill_counts))

    # Dynamic axis range (cap at 40 to keep it readable)
    MAX_SKILLS = int(min(40, max(skill_counts) if skill_counts else 40))
    BIN_COUNT  = MAX_SKILLS  # 1 bin per integer count

    fig = plt.figure(figsize=(12, 6), dpi=100)
    plt.hist(skill_counts, bins=BIN_COUNT, range=(0, MAX_SKILLS))
    ax = plt.gca()

    # Annotation
    ymax = ax.get_ylim()[1]
    xtext = MAX_SKILLS * 0.72
    ax.text(xtext, ymax*0.92, f"Mean   : {mean_count:.1f}", fontsize=10)
    ax.text(xtext, ymax*0.86, f"Median : {median_count:.1f}", fontsize=10)
    ax.text(xtext, ymax*0.80, f"Stdev  : {stdev_count:.1f}", fontsize=10)
    ax.text(xtext, ymax*0.74, f"p95    : {p95_count:.1f}", fontsize=10)
    ax.text(xtext, ymax*0.68, f"p99    : {p99_count:.1f}", fontsize=10)
    ax.text(xtext, ymax*0.62, f"% ≥1   : {nonzero_pct:.1f}%", fontsize=10)

    # Labels/format
    ax.set(xlim=(0, MAX_SKILLS),
           ylabel='Number of Job Postings',
           xlabel='Number of Extracted Skills per Job')
    ax.set_xticks(np.arange(0, MAX_SKILLS+1, max(1, MAX_SKILLS//8)))
    ax.set_title('Distribution of Extracted Skills per Job Posting (O*NET_ESCO)', fontsize=14)
    ax.grid(axis='y', alpha=0.5)
    plt.tight_layout()

    # Save
    plt.savefig('skill_count_distribution_combined.png')
    plt.close(fig)
    print("skill_count_distribution_combined.png")

skill_count_distribution_combined.png
